To achieve this, the class must implement `__enter__()` and `__exit()__` methods.

In [9]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM) -> None:
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

# Example use
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes: connection closed

Here's a breakdown of the code and its functionality:

**Imports:**

- `from socket import socket, AF_INET, SOCK_STREAM`: This line imports three objects from the `socket` module:
    - `socket`: This is the function used to create a socket object for network communication.
    - `AF_INET`: This constant represents the IPv4 address family (the most common type of IP address).
    - `SOCK_STREAM`: This constant represents the TCP socket type, which is used for reliable, connection-oriented communication.

**`LazyConnection` Class:**

- This class provides a way to manage network connections lazily. It establishes the connection only when needed, within the `with` block.
  - `__init__(self, address, family=AF_INET, type=SOCK_STREAM)`: The constructor takes three arguments:
    - `address`: A tuple containing the host (e.g., "www.python.org") and port (e.g., 80).
    - `family`: (Optional) The address family, defaults to `AF_INET` (IPv4).
    - `type`: (Optional) The socket type, defaults to `SOCK_STREAM` (TCP).
    It initializes internal variables to store the address, family, type, and a placeholder for the socket object (`self.sock`).
  - `__enter__(self)`: This method is called automatically when you enter the `with` block. It checks if the connection is already established (`self.sock is not None`). If not, it creates a socket using the stored address family and type (`socket(self.family, self.type)`), connects to the address (`self.sock.connect(self.address)`), and stores it in `self.sock`. Finally, it returns the socket object (`self.sock`) to be used within the `with` block.
  - `__exit__(self, exc_ty, exc_val, tb)`: This method is called automatically when the `with` block exits, either normally or due to an exception. It closes the socket connection if it's open (`self.sock.close()`) and sets the socket object back to `None`. It also takes three arguments (`exc_ty`, `exc_val`, and `tb`) for exception handling (not used in this specific example).

**Example Usage:**

- `from functools import partial`: This line imports the `partial` function from the `functools` module, which is used to create a partial function object.
- `conn = LazyConnection(('www.python.org', 80))`: This line creates an instance of the `LazyConnection` class, specifying the address as a tuple containing the hostname and port for www.python.org.
- `with conn as s`: This line uses the `LazyConnection` object with the `with` statement. Here's what happens:
    - `conn.__enter__()` is called, which creates the connection (if necessary) and assigns the socket object to `s`.
    - The code within the `with` block can now use the `s` object, which represents the established network connection.
    - `conn.__exit__()` is called automatically when the `with` block exits, which closes the connection (if it was open).

**Key Points:**

- The `LazyConnection` class promotes lazy connection creation, improving efficiency by establishing the connection only when actually needed within the `with` block.
- The context management protocol ensures that the connection is always closed properly, even if an exception occurs within the `with` block.

In [13]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM) -> None:
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

# Example use
from functools import partial

conn = LazyConnection(('www.python.org', 80))

with conn as s:
    # Send request headers (same as before)
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')

    # Receive and handle response data
    response = b''
    while True:
        data = s.recv(8192)  # Receive data in chunks
        if not data:
            break  # No more data to receive
        response += data

    # Process the received response data
    print(f"Received response:\n{response.decode()}")  # Decode and print

# Connection closed


Received response:
HTTP/1.1 301 Moved Permanently
Connection: close
Content-Length: 0
Server: Varnish
Retry-After: 0
Location: https://www.python.org/index.html
Accept-Ranges: bytes
Date: Fri, 10 May 2024 04:55:33 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdal2120129-DFW
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1715316934.725169,VS0,VE0
Strict-Transport-Security: max-age=63072000; includeSubDomains; preload


